In [1]:
import csv
import pandas as pd
import numpy as np
import pickle

In [2]:
import nltk
import string
import random
import warnings
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
lemmer = nltk.stem.WordNetLemmatizer()
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemTokens(tokens):
       return [lemmer.lemmatize(token,'v') for token in tokens if token not in set(stopwords.words('english'))]
def LemNormalize(text):
       return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))
def LemAllTokens(tokens):
       return [lemmer.lemmatize(token,'v') for token in tokens]
def LemAllNormalize(text):
       return LemAllTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [6]:
def getSentiment(sentence):
    loaded_model = pickle.load(open('SentimentModel.sav', 'rb'))
    Countvectorizer=pickle.load(open('Countvectorizer.pickle', 'rb'))
    indeces_to_delete=pickle.load(open('indeces_to_delete.pickle', 'rb'))
    getReady()
    X_new = Countvectorizer.transform(sentence)
    X_new_simple = np.delete(X_new.toarray(), indeces_to_delete, axis=1)
    return loaded_model.predict(X_new_simple)[0]

## this for using api to translate

# Put the credeintials in a file called service_account.json

In [8]:
"""Translates text into the target language.

Target must be an ISO 639-1 language code.
See https://g.co/cloud/translate/v2/translate-reference#supported_languages
"""
from google.cloud import translate_v2 as translate
def translate_text(target, text):


    translate_client = translate.Client.from_service_account_json(
        'service_account.json')



    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target)

    return result["translatedText"]
#     print(u"Text: {}".format(result["input"]))
#     print(u"Translation: {}".format(result["translatedText"]))
#     print(u"Detected source language: {}".format(result["detectedSourceLanguage"]))

# translate_text('en', "Hello")

In [35]:
greeting_inputs = ("اهلا","صباح الخير", "مساء الخير")
# greeting_responses = ["hey","You seems happy? Is that true?", "hey hows you?", "*nods*", "hello, how you doing", "hello", "Welcome, I am good and you?"]
goodbyes_responses = ["علي الرحب والسعه",'شكرا','شكرا جزيلا', 'شكرا لك', "مع السلامه","الي اللقاء"]
goodbyes_inputs = ('شكرا', 'السلامه','اللقاء')
sentiments={'sad':'حزين', 'afraid':"خائف قليلا", "confident":"واثق من نفسك","surprised":"متفاجئ قليلا"}
# def generate_greeting_response(greeting):
#     for token in greeting.split():
#         if token.lower() in greeting_inputs:
#             return random.choice(greeting_responses)
def generate_goodbyes_response(greeting):
    for token in greeting.split():
        if token.lower() in goodbyes_inputs:
            return random.choice(goodbyes_responses)
def wrong_sentiment(greeting):
    for token in greeting.split():
        if token.lower() =="لا":
            return True
def mapsentiment(sentiment):
    return sentiments[sentiment]


In [39]:
def generate_response(user_input,name):
    word_vectorizer = pickle.load(open(name+'_word_vectorizer.sav', 'rb'))
    all_word_vectors=pickle.load(open(name+'_all_word_vectors.pickle', 'rb'))
    utterances=pickle.load(open(name+'_utterance.pickle', 'rb'))
    prompts_sentences=pickle.load(open(name+'_prompts_sentences.pickle', 'rb'))
    translated_utterance=pickle.load(open(name+'_translated_utterance.pickle', 'rb'))
    tennisrobo_response = ''
    user_input_vector= word_vectorizer.transform([user_input])
    similar_prompt_vector_values = cosine_similarity(user_input_vector[0], all_word_vectors)
    similar_sentence_number = similar_prompt_vector_values.argsort()[0][-1]
                                      
    matched_vector = similar_prompt_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-1]
    if vector_matched != 0 and similar_sentence_number< len(utterances)-1:
        first = '' + utterances[similar_sentence_number+1]
        translated_first = '' + translated_utterance[similar_sentence_number+1]
        similar_sentence_second_number = similar_prompt_vector_values.argsort()[0][-2]
        second = '' + utterances[similar_sentence_second_number+1]
        translated_second = '' + translated_utterance[similar_sentence_second_number+1]
        similar_sentence_third_number = similar_prompt_vector_values.argsort()[0][-3]
        third = '' + utterances[similar_sentence_third_number+1]
        translated_third = '' + translated_utterance[similar_sentence_third_number+1]
#         print("first : ",first)
#         print(matched_vector[-1])
#         print("second : ",second)
#         print(matched_vector[-2])
#         print("third : ",third)
#         print(matched_vector[-3])
        Allreplies=[first,second,third]
        translated_Allreplies=[translated_first,translated_second,translated_third]
        Allprompts=[prompts_sentences[similar_sentence_number],prompts_sentences[similar_sentence_second_number],prompts_sentences[similar_sentence_third_number]]
        Countvectorizer = CountVectorizer(tokenizer = LemAllNormalize, ngram_range=(1,1))
        Countvectorizer.fit(Allprompts)
        counts_vector= Countvectorizer.transform(Allprompts)
        user_vector= Countvectorizer.transform([user_input])
        count_prompt_cosine_similarity = cosine_similarity(user_vector, counts_vector)
#         print(counts_vector[0])
        count_matched_vector = count_prompt_cosine_similarity.flatten()
        count_matched_vector.sort()
#         fi=count_prompt_cosine_similarity.argsort()[0][0]
#         print(Allprompts[fi],Allreplies[fi])
#         print(count_matched_vector[0])
#         si=count_prompt_cosine_similarity.argsort()[0][1]
#         print(Allprompts[si],Allreplies[si])
#         print(count_matched_vector[1])
#         ti=count_prompt_cosine_similarity.argsort()[0][2]
#         print(Allprompts[ti],Allreplies[ti])
#         print(count_matched_vector[2])
#         foi=count_prompt_cosine_similarity.argsort()[0][3]
# #         print(Allprompts[foi],Allreplies[foi])
# #         print(count_matched_vector[3])
        return translate_text('ar', Allreplies[count_prompt_cosine_similarity.argsort()[0][-1]])
    else:
        return" لا اعرف ماذا اقول ولكني اعرف انك "+ mapsentiment(name)

# Run this for using the chatbot

In [50]:
continue_dialogue = True
sentiment=None
print("فينتي: مرحبًا ، أنا صديقك فينتي. هل تريد التحدث معي قليلا؟ ")
while(continue_dialogue == True):
    
    human_text = input()
    human_text = human_text.lower()
    
    if human_text != 'bye':
        if generate_goodbyes_response(human_text) != None:
            continue_dialogue = False
            print("فينتي :" , generate_goodbyes_response(human_text))
        else:
            human_text=translate_text('en', human_text)
            if sentiment == None:
                sentiment= getSentiment([human_text])
                print("فينتي : ","يبدو أنك "+mapsentiment(sentiment)+" هل هذا صحيح؟")
                human_text = input()
                human_text = human_text.lower()
                human_text=translate_text('en', human_text)
                print("فينتي: حسنا ، هل تريد التحدث اكتر عن ذلك؟ ")
                human_text = input()
                human_text = human_text.lower()
                human_text=translate_text('en', human_text)
                if wrong_sentiment(human_text)==True:
                    sentiment= getSentiment([human_text])
#                 print("typing...")
                print("فينتي: ", generate_response(human_text,sentiment))
            else:
#                 print("typing...")
                print("فينتي: ", generate_response(human_text,sentiment))
    else:
        continue_dialogue = False
        print("فينتي: ","مع السلامه اعتني بنفسك جيدا")


فينتي: مرحبًا ، أنا صديقك فينتي. هل تريد التحدث معي قليلا؟ 
لقد عقدت العزيمه علي ان افقد وزني
فينتي :  يبدو أنك واثق من نفسك هل هذا صحيح؟
نعم
فينتي: حسنا ، هل تريد التحدث اكتر عن ذلك؟ 
انا حقا اريد ان افقد وزني
فينتي:  يمكنك. سوف يستغرق بعض المثابرة. كيف تسير الأمور حتى الآن؟
امارس الرياضه بانتظام واظن انني استطيع ان افقد وزني
فينتي:  أنا متأكد من أنه يمكنك فقط التفاني والجهد هو كل شيء!
هذا صحيح شكرا لتشجيعك
فينتي : علي الرحب والسعه
